In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image Generation with Imagen on Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/image_generation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/image_generation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/image_generation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


## Overview

[Imagen on Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/image/overview) brings Google's state of the art generative AI capabilities to application developers. With Imagen on Vertex AI, application developers can build next-generation AI products that transform their user's imagination into high quality visual assets, in seconds.

With Imagen, you can do the following:
- Generate novel images using only a text prompt (text-to-image generation).
- Edit an entire uploaded or generated image with a text prompt.
- Edit only parts of an uploaded or generated image using a mask area you define.
- Upscale existing, generated, or edited images.
- Fine-tune a model with a specific subject (for example, a specific handbag or shoe) for image generation.
- Get text descriptions of images with visual captioning.
- Get answers to a question about an image with Visual Question Answering (VQA).

This notebook focuses on **image generation** only. You can read more about image generation feature from Imagen [here](https://cloud.google.com/vertex-ai/docs/generative-ai/image/generate-images).


### Objectives

In this notebook, you will be exploring the image generation features of Imagen using the Vertex AI Python SDK. You will

- generate images using text prompts
- experiment with different parameters, such as:
    - increasing the number of images to be generated
    - fixing a seed number for reproducibility
    - influencing the output images using negative prompts

### Costs

- This notebook uses billable components of Google Cloud:
  - Vertex AI (Imagen)

- Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Getting Started

### Install Vertex AI SDK for Python

In [ ]:
! pip install --quiet --upgrade --user google-cloud-aiplatform

In [ ]:
import IPython
import time

### Load the image generation model

The model names from Vertex AI Imagen have two components: model name and version number. The naming convention follow this format: `<model-name>@<version-number>`. For example, `imagegeneration@001` represent the version **001** of the **imagetext** model.



In [ ]:
from vertexai.preview.vision_models import ImageGenerationModel

generation_model = ImageGenerationModel.from_pretrained("imagegeneration@006")

### Generate an image

The `generate_image` function can be used to generate images. All you need is a text prompt.

In [ ]:
prompt = "A beautiful ice princess with silver hair, fantasy concept art style"

response = generation_model.generate_images(
    prompt=prompt,
)

response.images[0].show()

Now, you have the power to generate the image you desire. Here are some example prompts to inspire you:
- A raccoon wearing formal clothes, wearing a top hat. Oil painting in the style of Vincent Van Gogh.
- A digital collage of famous works of art, all blended together into one cohesive image.
- A whimsical scene from a children's book, such as a tea party with talking animals.
- A futuristic cityscape with towering skyscrapers and flying cars.
- A studio photo of a modern armchair, dramatic lighting, warm.
- A surreal landscape of a city made of ice and snow.

In [ ]:
prompt = "A futuristic cityscape with towering skyscrapers and flying cars."  # @param {type:"string"}

response = generation_model.generate_images(prompt=prompt)

response.images[0].show()

###  Explore different parameters

The `generate_images` function accepts additional parameters that can be used to influence the generated images. The following sections will explore how to influence the output images through the use of those additional parameters.

In [ ]:
import math
import matplotlib.pyplot as plt


# An axuillary function to display images in grid
def display_images_in_grid(images):
    """Displays the provided images in a grid format. 4 images per row.

    Args:
        images: A list of PIL Image objects representing the images to display.
    """

    # Determine the number of rows and columns for the grid layout.
    nrows = math.ceil(len(images) / 4)  # Display at most 4 images per row
    ncols = min(len(images) + 1, 4)  # Adjust columns based on the number of images

    # Create a figure and axes for the grid layout.
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 6))

    for i, ax in enumerate(axes.flat):
        if i < len(images):
            # Display the image in the current axis.
            ax.imshow(images[i]._pil_image)

            # Adjust the axis aspect ratio to maintain image proportions.
            ax.set_aspect("equal")

            # Disable axis ticks for a cleaner appearance.
            ax.set_xticks([])
            ax.set_yticks([])
        else:
            # Hide empty subplots to avoid displaying blank axes.
            ax.axis("off")

    # Adjust the layout to minimize whitespace between subplots.
    plt.tight_layout()

    # Display the figure with the arranged images.
    plt.show()

#### number_of_images

You can use the `number_of_images` parameter to generate up to eight images.

In [ ]:
prompt = "a delicious bowl of pho from Vietnam"

response = generation_model.generate_images(
    prompt=prompt,
    number_of_images=4,
)

display_images_in_grid(response.images)

Try running the code a few time and observe the generate images. You will notice that the model generates a new set of images each time, even though the same prompt is used.

#### seed

With the `seed` parameter, you can influence the model to create the same output from the same input every time. Please note that the order of the generated images may still change.

In [ ]:
response = generation_model.generate_images(
    prompt=prompt,
    number_of_images=4,
    add_watermark=False,
    seed=42,
)

display_images_in_grid(response.images)

In [ ]:
response = generation_model.generate_images(
    prompt=prompt,
    number_of_images=4,
    add_watermark=False,
    seed=42,
)

display_images_in_grid(response.images)

As can be observed, the model produced an identical set of images after utilizing the seed parameter, although the order of the images varies.

#### negative_prompt

Objects you do not want to generate can be excluded using the `negative_prompt` parameter. "Bean sprout" was given as a negative prompt in the example below. As a result, despite using the same prompt and seed number, the model did not produce bean sprout in the images.

In [ ]:
response = generation_model.generate_images(
    prompt=prompt,
    number_of_images=4,
    add_watermark=False,
    seed=42,
    negative_prompt="bean sprout",
)

display_images_in_grid(response.images)

## Conclusion

You have explored the Imagen's image generation features through the Vertex AI Python SDK, including the additional parameters that influence image generation. The next step is to enhance your skills by exploring this [prompting guide](https://cloud.google.com/vertex-ai/docs/generative-ai/image/img-gen-prompt-guide?_ga=2.128324367.-2094800479.1701746552&_gac=1.219926379.1701161688.CjwKCAiAvJarBhA1EiwAGgZl0LFQUFOFZUxfNPlzjB4T00PDiLeCIEYfY-coLbX9eUfHKr_i8VbtSBoCEJQQAvD_BwE). Through practice, you will become proficient in the art of image prompting.

# Visual captioning with Imagen on Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/visual_captioning.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/visual_captioning.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/visual_captioning.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


## Overview

[Imagen on Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/image/overview) (image Generative AI) offers a variety of features:
- Image generation
- Image editing
- Visual captioning
- Visual question answering

This notebook focuses on **visual captioning** only.

[Visual captioning with Imagen on Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/image/image-captioning) can generate text descriptions of images. The model takes in an image as input and produces one or more text descriptions of the image as output. The generated text descriptions can be used for a variety of use cases:
- getting detailed metadata about images for storing and searching
- generating automated captioning to support accessibility use cases
- producing descriptions of products and visual assets

More information about Visual captioning with Imagen on  Vertex AI can be found in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/image/image-captioning).

### Objectives

In this notebook, you will learn how to use the Vertex AI Python SDK to:

- Generate image captions using the Imagen's visual captioning features

- Experiment with different parameters, such as:
    - number of captions to be generated
    - language of the generated captions
    - type and version of model that is used to generate the captions




### Costs

- This notebook uses billable components of Google Cloud:
  - Vertex AI (Imagen)

- Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Getting Started

### Load the image captioning model

The model names from Vertex AI Imagen have two components: model name and version number. The naming convention follow this format: `<model-name>@<version-number>`. For example, `imagetext@001` represent the version **001** of the **imagetext** model.



In [ ]:
from vertexai.preview.vision_models import ImageCaptioningModel

image_captioning_model = ImageCaptioningModel.from_pretrained("imagetext@001")

### Load the image file

To use the visual captioning model, you first need to create an `Image` class using the image file. The model only accepts `Image` class objects, so this is a necessary step before you can generate captions.

Moreover, [Visual Captioning with Imagen](https://cloud.google.com/vertex-ai/docs/generative-ai/image/image-captioning) only accepts specific image file formats (e.g. PNG, JPEG), and may have file size is limitations (e.g. 10 MB). You can find out specific details from [this official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/image/image-captioning#img-cap-rest).



In [ ]:
# Download an image from Google Cloud Storage
! gsutil cp "gs://github-repo/img/vision/google-cloud-next.jpeg" .

In [ ]:
from vertexai.preview.vision_models import Image

# Load the image file as Image object
cloud_next_image = Image.load_from_file("google-cloud-next.jpeg")
cloud_next_image.show()

###  Generate captions from the image

In this section, you will use the visual captioning model to generate text descriptions of an image.

In [ ]:
# Get a caption from the image
image_captioning_model.get_captions(
    image=cloud_next_image,
)

You can generate up to three captions from a single image by changing the `number_of_results` parameter from 1 to 3.

In [ ]:
# Get 3 captions from the image
image_captioning_model.get_captions(
    image=cloud_next_image,
    number_of_results=3,
)

### Generating captions in non-English languages

Visual captioning with Imagen on Vertex AI can generate captions in multiple languages as well. To generate a caption in a specific language, you can set the `language` parameter as one of the values:
- `en` - English
- `fr` - French
- `de` - German
- `it` - Italian
- `es` - Spanish

For a list of supported languages, check out the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/image/image-captioning#languages).

In [ ]:
# Get 3 image captions in French
image_captioning_model.get_captions(
    image=cloud_next_image,
    number_of_results=3,
    language="fr",
)

## Try it yourself

You can also try using the visual captioning model with images of your choice. If you need to download an image file, you can use the provided auxiliary function `download_image`.

Feel free to experiment with different images and model parameters to see how the results change.

In [ ]:
import os
import requests


def download_image(url):
    """Downloads an image from the specified URL."""

    # Send a get request to the url
    response = requests.get(url)

    # If the request is successful
    if response.status_code == 200:
        # Define image related variables
        image_path = os.path.basename(url)
        image_bytes = response.content
        image_type = response.headers["Content-Type"].split("/")[1]

        # Check for image type, currently only PNG or JPEG format are supported
        if image_type in ("png", "jpg", "jpeg"):
            # Write image data to a file
            with open(image_path, "wb") as f:
                f.write(image_bytes)
            return image_path
        else:
            raise Exception("Image can only be in PNG or JPEG format")

    else:
        raise Exception(f"Failed to download image from {url}")

In [ ]:
# Download an image
url = "https://t4.ftcdn.net/jpg/02/84/85/61/360_F_284856166_H5l84ry1V186YgMzZMXB0bhfJZTYAGCH.jpg"
download_image(url)

image_path = "360_F_284856166_H5l84ry1V186YgMzZMXB0bhfJZTYAGCH.jpg"

In [ ]:
# Load the newly downloaded image
user_image = Image.load_from_file(image_path)
user_image.show()

In [ ]:
# Generate the visual captions for the image
image_captioning_model.get_captions(
    image=user_image,
    number_of_results=3,
    language="en",
)

# Visual Question Answering (VQA) with Imagen on Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/visual_question_answering.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/visual_question_answering.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/visual_question_answering.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


## Overview

[Imagen on Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/image/overview) (image Generative AI) offers a variety of features:
- Image generation
- Image editing
- Visual captioning
- Visual question answering

This notebook focuses on **visual question answering** only.

[Visual question answering (VQA) with Imagen](https://cloud.google.com/vertex-ai/docs/generative-ai/image/visual-question-answering) can understand the content of an image and answer questions about it. The model takes in an image and a question as input, and then using the image as context to produce one or more answers to the question.

The visual question answering (VQA) can be used for a variety of use cases, including:
- assisting the visually impaired to gain more information about the images
- answering customer questions about products or services in the image
- creating interactive learning environment and providing interactive learning experiences

### Objectives

In this notebook, you will learn how to use the Vertex AI Python SDK to:

- Answering questions about images using the Imagen's visual question answering features

- Experiment with different parameters, such as:
    - number of answers to be provided by the model

### Costs

This tutorial uses billable components of Google Cloud:
- Vertex AI (Imagen)

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Getting Started

### Load the image question answering model

The model names from Vertex AI Imagen have two components: model name and version number. The naming convention follow this format: `<model-name>@<version-number>`. For example, `imagetext@001` represents the version **001** of the **imagetext** model.



In [ ]:
from vertexai.preview.vision_models import ImageQnAModel

image_qna_model = ImageQnAModel.from_pretrained("imagetext@001")

### Load the image file

To use the image question answering model, you first need to create an `Image` class using the image file. The model only accepts `Image` class objects, so this is a necessary step before you can ask questions.


Addtinally, [Visual Question Answering with Imagen](https://cloud.google.com/vertex-ai/docs/generative-ai/image/visual-question-answering#img-vqa-rest) only accepts specific image file formats (e.g. PNG, JPEG), and may have file size is limitations (e.g. 10 MB). You can find out specific details from the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/image/visual-question-answering#img-vqa-rest).

In [ ]:
from vertexai.preview.vision_models import Image

# Load the image file as Image object
cloud_next_image = Image.load_from_file("google-cloud-next.jpeg")
cloud_next_image.show()

### Ask questions about the image

Now ask questions about the image using the model:

In [ ]:
# Ask a question about the image
image_qna_model.ask_question(
    image=cloud_next_image, question="What is happening in this image?"
)

In [ ]:
# Ask a follow up question about the image
image_qna_model.ask_question(
    image=cloud_next_image, question="What are the people in the image doing?"
)

You can get up to three answers from a single image by changing the `number_of_results` parameter from 1 to 3.

In [ ]:
# Get 3 answers from the image
image_qna_model.ask_question(
    image=cloud_next_image,
    question="What are the people in the image doing?",
    number_of_results=3,
)

## Try it yourself

You can also try using the visual question answering model with images of your choice. If you need to download an image file, you can use the provided auxiliary function `download_image`.

Feel free to experiment with different images and model parameters to see how the results change.

In [ ]:
# Download an image
url = "https://storage.googleapis.com/gweb-cloudblog-publish/images/GettyImages-871168786.max-2600x2600.jpg"
image_path = download_image(url)

# Load the newly downloaded image
user_image = Image.load_from_file(image_path)
user_image.show()

In [ ]:
# Ask a question about the image
image_qna_model.ask_question(
    image=user_image,
    question="What is happening in this photo?",
    number_of_results=3,
)

In [ ]:
# Ask a question about the image
image_qna_model.ask_question(
    image=user_image,
    question="What advertising channels would this image be suitable for?",
    number_of_results=3,
)

In [ ]:
# Ask a question about the image
image_qna_model.ask_question(
    image=user_image,
    question="What type of insects could live in this area?",
    number_of_results=3,
)